# 📘 Galileo + OpenTelemetry Tracing Guide (Python)

This notebook demonstrates how to send OpenTelemetry traces to Galileo using OpenInference.

🔗 See full documentation [here](https://v2galileo.mintlify.app/guides/logs/otel)

In [ ]:
!pip install -U \
    crewai \
    crewai_tools \
    openinference-instrumentation-crewai \
    openinference-instrumentation-litellm \
    openinference-instrumentation-langchain \
    opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-api \
    "opentelemetry-proto>=1.12.0"  -q

In [ ]:
GALILEO_API_KEY = "YOUR_API_KEY"
PROJECT_NAME = "YOUR-PROJECT-NAME"
LOG_STREAM_NAME = "my_log_stream"

## 🔐 Authentication and Header Setup

In [ ]:
import os

headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": PROJECT_NAME,
    "logstream": LOG_STREAM_NAME,
}
# Format headers as a comma-separated key=value string
# Example: Galileo-API-Key=abc123,project_name=my_project,log_stream_name=my_run
os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'] = ",".join([f"{k}={v}" for k, v in headers.items()])

## ⚙️ OpenTelemetry Setup

In [ ]:
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry import trace as trace_api
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from openinference.instrumentation.crewai import CrewAIInstrumentor
from openinference.instrumentation.litellm import LiteLLMInstrumentor

# OTEL endpoint for Galileo
endpoint = "https://app.galileo.ai/api/galileo/otel/traces"

# Setup tracer provider
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint)))

# Optional: log locally
tracer_provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))

# Register the provider
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# CrewAI Instrument (0.63.0 and above uses litellm)
LiteLLMInstrumentor().instrument(tracer_provider=tracer_provider)
CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

print("CrewAI OTEL tracing initialized.")


OTEL tracing initialized.


# Demo

In [ ]:
import os
try:
  if not os.environ.get("OPENAI_API_KEY"):
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
  print("Please set the  OPENAI_API_KEY variable")
from crewai import Agent, Crew, Process, Task

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    # You can pass an optional llm attribute specifying what model you wanna use.
    # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer], tasks=[task1, task2], verbose=1, process=Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)